In [1]:
!pip install transformers

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# repo_name = "TayyMem/Grammar_Spelling_Correction"
repo_name = "TayyMem/ProjectGG_Testing3"

# Load model and tokenizer from Hugging Face
tokenizer = T5Tokenizer.from_pretrained(repo_name)
model = T5ForConditionalGeneration.from_pretrained(repo_name)

print("Model loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Model loaded successfully!


In [30]:
from nltk.translate.bleu_score import sentence_bleu
!pip install sacrebleu
import sacrebleu
!pip install rouge_score

https://huggingface.co/docs/transformers/v4.15.0/en/model_sharing#add-a-model-card

In [41]:
# Define the device
# If a GPU is available, it will use the GPU. Otherwise, it will use the CPU.
# Load the ROUGE evaluation metric
rouge = evaluate.load('rouge')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

example_sentence = [
   "She don’t likes to goes to the park on weekend.",
   "the childs was playing in the garden when it start to raining.",
   "I has finish my homework and now I am going to watch a movie.",
   "Their is too many people in the mall today, its very crowded.",
   "If he studied more harder, he will passed the exam.",
   "She never been to this restaurant before.",
   "Me and my friend go to cinema yesterday night.",
   "They buyed a new car because the old one break down.",
   "He forgetted to bring his homeworks to school.",
   "The weather is geting colder as winter approaching.",
   "I need buy some vegetable for dinner tonite.",
   "We will visit our grandparent on next weekend.",
   "She was read a book when I arrive.",
   "He is learning to plays piano this years.",
   "They moved to a new house last months.",
   ]


#tokenizing the example sentence
testing_input = tokenizer(
    example_sentence,
    max_length=256,
    truncation=True,
    padding=True,
    return_tensors='pt').to(device)

#generate output
testing_output = model.generate(
    input_ids= testing_input["input_ids"],
    attention_mask= testing_input["attention_mask"],
    num_beams=5,  # Better search for the best correction
    max_length=256) # A way to predict data

#decode the output
corrected_sentences = [tokenizer.decode(output, skip_special_tokens=True) for output in testing_output]

# Tokenize predictions (bleu cant process string. so need to split model's output)
tokenized_corrected_sentences = [sentence.split() for sentence in corrected_sentences]

# Define your desired weights (example: higher weight for bi-grams)
weights = (.4, 0.4, 0.2, 0)  # Weights for uni-gram, bi-gram, tri-gram, and 4-gram

# the answer for my incorrect sentences. had to split as bleu understand this format only
reference_sentences = [
   ["She", "doesn't", "like", "to", "go", "to", "the", "park", "on", "weekends."],
   ["The", "children", "were", "playing", "in", "the", "garden", "when", "it", "started", "raining."],
   ["I", "have", "finished", "my", "homework,", "and", "now", "I", "am", "going", "to", "watch", "a", "movie."],
   ["There", "are", "too", "many", "people", "in", "the", "mall", "today;", "it’s", "very", "crowded."],
   ["If", "he", "studied", "harder,", "he", "would", "pass", "the", "exam."],
   ["She", "has", "never", "been", "to", "this", "restaurant", "before."],
   ["My", "friend", "and", "I", "went", "to", "the", "cinema", "last", "night."],
   ["They", "bought", "a", "new", "car", "because", "the", "old", "one", "broke", "down."],
   ["He", "forgot", "to", "bring", "his", "homework", "to", "school."],
   ["The", "weather", "is", "getting", "colder", "as", "winter", "approaches."],
   ["I", "need", "to", "buy", "some", "vegetables", "for", "dinner", "tonight."],
   ["We", "will", "visit", "our", "grandparents", "next", "weekend."],
   ["She", "was", "reading", "a", "book", "when", "I", "arrived."],
   ["He", "is", "learning", "to", "play", "the", "piano", "this", "year."],
   ["They", "moved", "to", "a", "new", "house", "last", "month."]
]

reference_sentences_chrf = [
   "She  doesn't  like  to  go  to  the  park  on  weekends.",
   "The  children  were  playing  in  the  garden  when  it  started  raining.",
   "I  have  finished  my  homework   and  now  I  am  going  to  watch  a  movie.",
   "There  are  too  many  people  in  the  mall  today;  it’s  very  crowded.",
   "If  he  studied  harder   he  would  pass  the  exam.",
   "She  has  never  been  to  this  restaurant  before.",
   "My  friend  and  I  went  to  the  cinema  last  night.",
   "They  bought  a  new  car  because  the  old  one  broke  down.",
   "He  forgot  to  bring  his  homework  to  school.",
   "The  weather  is  getting  colder  as  winter  approaches.",
   "I  need  to  buy  some  vegetables  for  dinner  tonight.",
   "We  will  visit  our  grandparents  next  weekend.",
   "She  was  reading  a  book  when  I  arrived.",
   "He  is  learning  to  play  the  piano  this  year.",
   "They  moved  to  a  new  house  last  month.",
]

for i in range(len(example_sentence)):
  a = [reference_sentences[i]]  # BLEU expects a list of references
  b = tokenized_corrected_sentences[i]  # Tokenized prediction
  #bleu score calculation, must use tokenized sentences.
  score = sentence_bleu(a, b, weights=weights)
  #chrf score calculation,  can use sentences.
  chrf_score = sacrebleu.sentence_chrf(example_sentence[i], [reference_sentences_chrf[i]])
  #rouge score calculation and rouge score doesn't need to use tokenized sentences.
  rouge_score = rouge.compute(predictions=[example_sentence[i]], references=[reference_sentences_chrf[i]])



  print(f"Incorrect: {example_sentence[i]}")
  print(f"Corrected: {corrected_sentences[i]}")
  print(f"BLEU Score: {score:.4f}")  # 4f is getting 4 decimal values.
  print(f"CHRF Score: {chrf_score.score:.4f}")
  #print(f"ROUGE Score: {rouge_score:.4f}\n")       #this doesn't give result as rouge has format of []. so have to get each.
  print(f"ROUGE1 Score: {rouge_score['rouge1']:.4f}")
  print(f"ROUGE2 Score: {rouge_score['rouge2']:.4f}")
  print(f"ROUGEL Score: {rouge_score['rougeL']:.4f}")
  print(f"rougeLsum Score: {rouge_score['rougeLsum']:.4f}\n")

Incorrect: She don’t likes to goes to the park on weekend.
Corrected: She doesn’t like going to the park on weekend.
BLEU Score: 0.4000
CHRF Score: 65.2483
ROUGE1 Score: 0.6364
ROUGE2 Score: 0.3000
ROUGEL Score: 0.6364
rougeLsum Score: 0.6364

Incorrect: the childs was playing in the garden when it start to raining.
Corrected: the childs was playing in the garden when it started to rain.
BLEU Score: 0.5506
CHRF Score: 71.7639
ROUGE1 Score: 0.6957
ROUGE2 Score: 0.4762
ROUGEL Score: 0.6957
rougeLsum Score: 0.6957

Incorrect: I has finish my homework and now I am going to watch a movie.
Corrected: I have finished my homework and now i am going to watch a movie.
BLEU Score: 0.7065
CHRF Score: 83.2798
ROUGE1 Score: 0.8571
ROUGE2 Score: 0.7692
ROUGEL Score: 0.8571
rougeLsum Score: 0.8571

Incorrect: Their is too many people in the mall today, its very crowded.
Corrected: There are too many people in the mall today; it is very crowded.
BLEU Score: 0.7616
CHRF Score: 74.3892
ROUGE1 Score: 0.72

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Incorrect: I need buy some vegetable for dinner tonite.
Corrected: I need buy some vegetable for dinner tonite.
BLEU Score: 0.0000
CHRF Score: 70.6482
ROUGE1 Score: 0.7059
ROUGE2 Score: 0.4000
ROUGEL Score: 0.7059
rougeLsum Score: 0.7059

Incorrect: We will visit our grandparent on next weekend.
Corrected: We will visit our grandparent next weekend.
BLEU Score: 0.6656
CHRF Score: 89.3823
ROUGE1 Score: 0.8000
ROUGE2 Score: 0.6154
ROUGEL Score: 0.8000
rougeLsum Score: 0.8000

Incorrect: She was read a book when I arrive.
Corrected: She read a book when i arrived.
BLEU Score: 0.3539
CHRF Score: 75.8900
ROUGE1 Score: 0.7500
ROUGE2 Score: 0.5714
ROUGEL Score: 0.7500
rougeLsum Score: 0.7500

Incorrect: He is learning to plays piano this years.
Corrected: He is learning to play piano these years.
BLEU Score: 0.5474
CHRF Score: 80.6386
ROUGE1 Score: 0.7059
ROUGE2 Score: 0.5333
ROUGEL Score: 0.7059
rougeLsum Score: 0.7059

Incorrect: They moved to a new house last months.
Corrected: They moved 

AttributeError: 'numpy.float64' object has no attribute 'precision'